# Method of transforming whole image to png

In [2]:
import openslide
from tqdm import tqdm
from PIL import Image
import numpy as np
import os

def resize_ndpi_image(ndpi_path, output_path, target_size=(224, 224)):
    slide = openslide.OpenSlide(ndpi_path)
    
    # Choose a level with reasonable resolution
    level = slide.get_best_level_for_downsample(32)
    thumbnail = slide.read_region((0, 0), level, slide.level_dimensions[level])
    
    # Convert to RGB and resize
    thumbnail = thumbnail.convert("RGB")
    resized = thumbnail.resize(target_size, Image.Resampling.LANCZOS)

    
    resized.save(output_path)

In [ ]:
input_dir = "/Volumes/Maxtor/CONTINUACION Basurto y Cruces"
output_dir = "/Users/cameronpreasmyer/Desktop/Images_png/Basurto and Cruces"

for filename in os.listdir(input_dir):
    if filename.endswith(".ndpi"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace(".ndpi", ".png"))
        resize_ndpi_image(input_path, output_path)


# Sorting Algo 

In [22]:
import os
import shutil

keywords = ["PIR", "PBP", "MITF", "HE", "ANX", "BRAF", "BCL2", "BCL3"]
source_folder = "/Users/cameronpreasmyer/Desktop/Images_png/Basurto and Cruces"
destination_root = "/Users/cameronpreasmyer/Desktop/Images_png/ROOT"


for filename in os.listdir(source_folder):
    upper_name = filename.upper()
    for keyword in keywords:
        if keyword in upper_name:
            keyword_folder = os.path.join(destination_root, keyword)
            os.makedirs(keyword_folder, exist_ok=True)
            src_path = os.path.join(source_folder, filename)
            dst_path = os.path.join(keyword_folder, filename)
            shutil.move(src_path, dst_path)
            print(f"Moved {filename} to {keyword_folder}")
            break  # Avoid moving the same file to multiple folders



Moved EL-B142B in situ-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-B125-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-B158NID3-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-C011-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-C010-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-B158NID2-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-B153-BCL2.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL2
Moved EL-B176-PBP.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/PBP
Moved EL-B176-ANX.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/ANX
Moved EL-C017-BCL3 - 2018-07-12 15.46.58.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-C018-PBP.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/PBP
Moved EL-B209NID-BCL3.png to /Users/cameronpreasmyer/Desktop/Images_png/ROOT/BCL3
Moved EL-C017-MI

# Method of transforming whole image to Tiles 

In [3]:

def is_informative(tile, threshold=0.92):
    '''
    Returns True if the tile is informative, False otherwise.
    '''
    arr = np.array(tile)
    gray = np.mean(arr, axis=-1)
    white_pixels = np.sum(gray > 224)
    total_pixels = gray.size
    return (white_pixels / total_pixels) < threshold



In [ ]:
def extract_tiles(slide_path, output_dir, tile_size=(512, 512), stride=512):
    slide = openslide.OpenSlide(slide_path)
    width, height = slide.dimensions
    os.makedirs(output_dir, exist_ok=True)

    # Calculate total number of tiles
    total_tiles = ((height - tile_size[1]) // stride + 1) * ((width - tile_size[0]) // stride + 1)
    count = 0

    with tqdm(total=total_tiles, desc="Extracting tiles") as pbar:
        for y in range(0, height, stride):
            for x in range(0, width, stride):
                if x + tile_size[0] <= width and y + tile_size[1] <= height:
                    tile = slide.read_region((x, y), 0, tile_size).convert("RGB")
                    if is_informative(tile):
                        tile.save(os.path.join(output_dir, f"tile_{count}.png"))
                        count += 1
                    pbar.update(1)

In [ ]:
# Testing levels of informative-ness to tweak above threshold and gray value

is_informative(Image.open("/Users/cameronpreasmyer/Desktop/outputtest/tiles/tile_69.png"))

np.True_

In [145]:
input_ndpi = "/Users/cameronpreasmyer/Desktop/Inputtest/EL-C021-ANX.ndpi"
output_tiles = "/Users/cameronpreasmyer/Desktop/outputtest/tiles"

extract_tiles(input_ndpi, output_tiles, tile_size=(512, 512), stride=512)


Extracting tiles:  17%|█▋        | 4269/25740 [03:30<17:38, 20.29it/s]  


KeyboardInterrupt: 